In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import nglview as nv
import py3Dmol

from proteome import protein
from proteome.models.genie import config
from proteome.models.genie.modeling import GenieForStructureDesign, GENIE_MODEL_CONFIGS
from proteome.models.proteinmpnn.modeling import ProteinMPNNForSequenceDesign
from proteome.models.omegafold.modeling import OmegaFoldForFolding

In [ ]:
structure_designer = GenieForStructureDesign(model_name="genie_l_256", random_seed=0)
sequence_designer = ProteinMPNNForSequenceDesign("ca_only_model-20", random_seed=37)
folder = OmegaFoldForFolding()

In [ ]:
structure = structure_designer.design_structure(config.InferenceConfig(seq_len=100))

In [ ]:
view = nv.show_text(protein.to_pdb(structure))
view

In [ ]:
sequence, score = sequence_designer.design_sequence(protein.to_ca_only_protein(structure))
print(f"Sequence: {sequence} with global_score {score}")

In [ ]:
predicted_protein, confidence = folder.fold(sequence)
result_pdb = protein.to_pdb(predicted_protein)

In [ ]:
PLDDT_BANDS = [
  (0, 50, '#FF7D45'),
  (50, 70, '#FFDB13'),
  (70, 90, '#65CBF3'),
  (90, 100, '#0053D6')
]
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(result_pdb)

color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}

style['stick'] = {}

view.setStyle({'model': -1}, style)
view.zoomTo()